<a href="https://colab.research.google.com/github/farroshsy/CollaborativeFiltering-BuildingReccomender_BigData/blob/main/CollaborativeFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Big Data Assignment Week 9**
## Filtering Collaborative
---

Name: Farros Hilmi Syafei 
<br>
Student ID: 5025201012
<br>
Class: Big Data A
<br>
Lecturer: Abdul Munif, S.Kom., M.Sc.


reference: https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

## Install & Initialization

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 11.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=6d3e3347f9d64a857fe564111a5c526dab6350523833a1b8090ca237e0b8dd41
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [13]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row, SparkSession

In [14]:
# SparkSession Initialization
spark = SparkSession.builder \
    .master("local") \
    .appName("MovieLens") \
    .getOrCreate()

In [15]:
# Read data from a text file and separate elements of each line
lines = spark.read.text("./sample_data/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))

In [16]:
# Convert data into a DataFrame with userId, movieId, rating, and timestamp columns
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))

# Split the data into training (80%) and testing (20%) sets
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

## Build Recomendation model using ALS

In [19]:
# Initialize the parameters to be tried
max_iters = [5, 10, 15]
reg_params = [0.1, 0.5, 1.0]

# Dictionary to store RMSE results
results = {}

In [20]:
# Loop for every combination of maxIter and regParam
for max_iter in max_iters:
    for reg_param in reg_params:
        # Build the recommendation model using ALS on the training data
        # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
        als = ALS(maxIter=max_iter, regParam=reg_param, userCol="userId", itemCol="movieId", ratingCol="rating",
                  coldStartStrategy="drop")
        model = als.fit(training)

        # Evaluate the model by computing the RMSE on the test data
        predictions = model.transform(test)
        evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                        predictionCol="prediction")
        rmse = evaluator.evaluate(predictions)

        # Save the RMSE result in the dictionary
        results[(max_iter, reg_param)] = rmse
        print(f"Root-mean-square error for maxIter={max_iter}, regParam={reg_param} = {rmse}")

Root-mean-square error for maxIter=5, regParam=0.1 = 1.0884613542905341
Root-mean-square error for maxIter=5, regParam=0.5 = 1.2079734043522738
Root-mean-square error for maxIter=5, regParam=1.0 = 1.4967499541655533
Root-mean-square error for maxIter=10, regParam=0.1 = 0.9908285031041218
Root-mean-square error for maxIter=10, regParam=0.5 = 1.2038556335334538
Root-mean-square error for maxIter=10, regParam=1.0 = 1.4967146454568323
Root-mean-square error for maxIter=15, regParam=0.1 = 0.9786344420028297
Root-mean-square error for maxIter=15, regParam=0.5 = 1.2032519694174355
Root-mean-square error for maxIter=15, regParam=1.0 = 1.4967146781966396


In [21]:
# Find the hyperparameter combination with the lowest RMSE
best_params = min(results, key=results.get)
best_rmse = results[best_params]
print(f"\nBest hyperparameters: maxIter={best_params[0]}, regParam={best_params[1]} with RMSE={best_rmse}")


Best hyperparameters: maxIter=15, regParam=0.1 with RMSE=0.9786344420028297


## Generate Movie Recomendation

In [22]:
# Train the model with the best hyperparameters
best_als = ALS(maxIter=best_params[0], regParam=best_params[1], userCol="userId", itemCol="movieId", ratingCol="rating",
               coldStartStrategy="drop")
best_model = best_als.fit(training)

## Print Result and Show Ouput

In [23]:
# Generate top 10 movie recommendations for each user
userRecs = best_model.recommendForAllUsers(10)
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{22, 3.726203}, ...|
|    10|[{49, 2.7171586},...|
|     0|[{2, 2.588266}, {...|
|     1|[{62, 2.9802883},...|
|    21|[{29, 4.2106977},...|
|    11|[{32, 4.717435}, ...|
|    12|[{64, 4.2527}, {1...|
|    22|[{88, 4.5162344},...|
|     2|[{93, 4.6412106},...|
|    13|[{93, 3.115012}, ...|
|     3|[{30, 4.2603374},...|
|    23|[{32, 4.7912803},...|
|     4|[{29, 3.5301394},...|
|    24|[{69, 4.468046}, ...|
|    14|[{29, 4.5012035},...|
|     5|[{46, 3.7628384},...|
|    15|[{46, 4.230317}, ...|
|    25|[{46, 3.763657}, ...|
|    26|[{22, 4.779094}, ...|
|     6|[{25, 4.231402}, ...|
+------+--------------------+
only showing top 20 rows



In [24]:
# Generate top 10 user recommendations for each movie
movieRecs = best_model.recommendForAllItems(10)
movieRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     20|[{17, 4.32391}, {...|
|     40|[{2, 2.313482}, {...|
|     10|[{23, 3.7053993},...|
|     50|[{23, 3.708891}, ...|
|     80|[{3, 3.2182882}, ...|
|     70|[{8, 3.2199912}, ...|
|     60|[{22, 2.6367269},...|
|     90|[{16, 4.5834417},...|
|     30|[{11, 4.6718946},...|
|      0|[{28, 2.816168}, ...|
|     31|[{12, 3.1125932},...|
|     81|[{11, 3.3977723},...|
|     91|[{23, 2.6864283},...|
|      1|[{25, 2.9285612},...|
|     41|[{8, 3.5585325}, ...|
|     61|[{6, 2.2451425}, ...|
|     51|[{26, 4.0095773},...|
|     21|[{26, 2.7833862},...|
|     11|[{18, 3.3367631},...|
|     71|[{25, 3.002106}, ...|
+-------+--------------------+
only showing top 20 rows



In [25]:
# Generate top 10 movie recommendations for a specific set of users
users = ratings.select(best_als.getUserCol()).distinct().limit(3)
userSubsetRecs = best_model.recommendForUserSubset(users, 10)
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{22, 4.779094}, ...|
|    19|[{90, 3.4500241},...|
|    29|[{46, 4.4573317},...|
+------+--------------------+



In [26]:
# Generate top 10 user recommendations for a specific set of movies
movies = ratings.select(best_als.getItemCol()).distinct().limit(3)
movieSubSetRecs = best_model.recommendForItemSubset(movies, 10)
movieSubSetRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     65|[{23, 4.2237153},...|
|     26|[{28, 2.5338242},...|
|     29|[{8, 4.7771497}, ...|
+-------+--------------------+



## Summary

The following sections describe the implemented steps:

* Data Loading and Preprocessing: </br>
We loaded the MovieLens dataset, processed it, and split it into training and test sets.

* Hyperparameter Tuning: </br>
We tried different combinations of maxIter and regParam values, evaluated the models on the test set, and stored the RMSE results in a dictionary.

* Model Selection: </br>
We identified the best hyperparameter combination based on the lowest RMSE and displayed the results.

* Generating Recommendations: </br>
We trained the model using the best hyperparameters and generated recommendations for all users, all movies, a subset of users, and a subset of movies.

The output shows the top 10 recommendations for each user, movie, and the specified subsets.